In [85]:
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
os.chdir('/Users/xiaodiu/Documents/github/JudgeSentences/')

In [5]:
df = pd.DataFrame()
itr = pd.read_stata('rawdata/sentences/opafy00nid.dta',iterator=True)
chunk = itr.get_chunk(10000)
df = df.append(chunk, ignore_index=True)

In [4]:

from glob import glob

import pandas as pd
import re


folders = glob('*')

num = 0
for folder in folders:        
    members = glob(folder + '/*txt')
    num += len(members)
    
print "total %d" %(num)

total 251963


In [15]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
casefreq_akd = pd.read_pickle('rawdata/case_level/casefreqs-akd.pkl')
id2word = pd.read_pickle('rawdata/id2word-2.pkl')
word2id = pd.read_pickle('rawdata/word2id-2.pkl')
df_Holger = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2.dta')
df_Holger_demean = pd.read_stata('rawdata/Holger_matched_USSC_TRAC2_include_demean.dta')
df_Trac_USSC_Judges = pd.read_stata('rawdata/Trac_USSC_JudgesBIOs.dta')
docfreq_akd = pd.read_pickle('rawdata/text/docfreqs-akd.pkl')

In [114]:
case2judge = pd.read_pickle('rawdata/cluster2songername.pkl')
judge2case = dict(zip(case2judge.values(),case2judge.keys()))
files = glob('rawdata/case_level/*.pkl')
num = len(files)
judge2count_bigram = Counter()
i = 1
for file in files:
    #print('file :'+ file)
    case_freq = pd.read_pickle(file)
    for k,v in case_freq.items():
        k = int(k)
        if k in case2judge:
            judge = case2judge[k]
            #print('judge_name:' + judge)
            if(judge2count_bigram[judge] == 0):
                judge2count_bigram[judge] = Counter()
            judge2count_bigram[judge] += v
    if(i % 20 == 0 ):
        print('finished : %s' % (i))
    i += 1

finished : 20
finished : 40
finished : 60
finished : 80
finished : 100


In [118]:
judge2exist_bigram = Counter()
for k, v in judge2count_bigram.items():
    judge2exist_bigram[k] = Counter(v.keys())

In [120]:
pd.to_pickle(judge2exist_bigram,
             'rawdata/judge2exist_bigram.pkl')
pd.to_pickle(judge2count_bigram,
             'rawdata/judge2count_bigram.pkl')